In [2]:
import re
import timeit
from time import perf_counter
import pandas as pd
from collections import defaultdict
import csv


In [18]:
output_df = pd.read_csv('output.csv')

In [19]:
top_10_palabras = ['science', 'data', 'learning', 'research', 'computer', 'ai', 'systems', 'applications', 'intelligence', 'field']


In [20]:
def medir_tiempo_findall_timeit(patron, texto, iteraciones=10000):
    tiempo = 0
    for _ in range(iteraciones):
        tiempo += timeit.timeit(lambda: re.findall(patron, texto), number=1)
    return tiempo

In [21]:
def medir_tiempo_finditer_timeit(patron, texto, iteraciones=10000):
    tiempo = 0
    for _ in range(iteraciones):
        tiempo += timeit.timeit(lambda: re.finditer(patron, texto), number=1)
    return tiempo


In [22]:
def medir_tiempo_search_timeit(patron, texto, iteraciones=10000):
    tiempo = 0
    for _ in range(iteraciones):
        tiempo += timeit.timeit(lambda: re.search(patron, texto), number=1)
    return tiempo


In [23]:
def medir_tiempo_findall_perf(patron, texto, iteraciones=10000):
    inicio = perf_counter()
    for _ in range(iteraciones):
        re.findall(patron, texto)
    fin = perf_counter()
    return fin - inicio

In [24]:
def medir_tiempo_finditer_perf(patron, texto, iteraciones=10000):
    inicio = perf_counter()
    for _ in range(iteraciones):
        re.finditer(patron, texto)
    fin = perf_counter()
    return fin - inicio

In [25]:
def medir_tiempo_search_perf(patron, texto, iteraciones=10000):
    inicio = perf_counter()
    for _ in range(iteraciones):
        re.search(patron, texto)
    fin = perf_counter()
    return fin - inicio

In [26]:
resultados = []

In [27]:
for palabra in top_10_palabras:
    patron = fr'\b{palabra}\w*'
    for index, row in output_df.iterrows():
        texto = row['abstractText']
        # Medir tiempos usando timeit
        tiempo_findall_timeit = medir_tiempo_findall_timeit(patron, texto)
        tiempo_finditer_timeit = medir_tiempo_finditer_timeit(patron, texto)
        tiempo_search_timeit = medir_tiempo_search_timeit(patron, texto)
        # Medir tiempos usando perf_counter
        tiempo_findall_perf = medir_tiempo_findall_perf(patron, texto)
        tiempo_finditer_perf = medir_tiempo_finditer_perf(patron, texto)
        tiempo_search_perf = medir_tiempo_search_perf(patron, texto)

        resultados.append({'palabra': palabra, 'metodo': 'findall', 'medicion': 'timeit', 'tiempo': tiempo_findall_timeit, 'texto_index': index})
        resultados.append({'palabra': palabra, 'metodo': 'finditer', 'medicion': 'timeit', 'tiempo': tiempo_finditer_timeit, 'texto_index': index})
        resultados.append({'palabra': palabra, 'metodo': 'search', 'medicion': 'timeit', 'tiempo': tiempo_search_timeit, 'texto_index': index})
        resultados.append({'palabra': palabra, 'metodo': 'findall', 'medicion': 'perf_counter', 'tiempo': tiempo_findall_perf, 'texto_index': index})
        resultados.append({'palabra': palabra, 'metodo': 'finditer', 'medicion': 'perf_counter', 'tiempo': tiempo_finditer_perf, 'texto_index': index})
        resultados.append({'palabra': palabra, 'metodo': 'search', 'medicion': 'perf_counter', 'tiempo': tiempo_search_perf, 'texto_index': index})

In [28]:
resultados_df = pd.DataFrame(resultados)
resultados_df.to_csv('resultados_tiempos.csv', index=False)

In [29]:
tiempos = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))
promedios_generales = defaultdict(lambda: defaultdict(float))

### Lectura datos del csv

In [30]:
with open('resultados_tiempos.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        palabra = row['palabra']
        algoritmo = row['metodo']
        tipo_medicion = row['medicion']
        tiempo = float(row['tiempo'])
        tiempos[palabra][algoritmo][tipo_medicion].append(tiempo)

In [31]:
resultados = []
promedios_generales = defaultdict(lambda: defaultdict(float))

### Promedio por palabras

In [32]:
for palabra, algoritmos in tiempos.items():
    for algoritmo, mediciones in algoritmos.items():
        for tipo_medicion, tiempos_lista in mediciones.items():
            promedio_tiempo = sum(tiempos_lista) / len(tiempos_lista)
            resultados.append({
                'Palabra': palabra,
                'Algoritmo': algoritmo,
                'Tipo de Medición': tipo_medicion,
                'Promedio Tiempo': promedio_tiempo
            })
            promedios_generales[algoritmo][tipo_medicion] += promedio_tiempo

### Promedio por Algoritmo

In [33]:
promedios_generales_resultados = []
for algoritmo, mediciones in promedios_generales.items():
    for tipo_medicion, tiempo_total in mediciones.items():
        cantidad_palabras = len(tiempos)  # Número total de palabras
        promedio_general = tiempo_total / cantidad_palabras
        promedios_generales_resultados.append({
            'Algoritmo': algoritmo,
            'Tipo de Medición': tipo_medicion,
            'Promedio General': promedio_general
        })

### Exportar Datos

In [34]:
csv_file = 'resultados_palabras.csv'
csv_columns = ['Palabra', 'Algoritmo', 'Tipo de Medición', 'Promedio Tiempo']

In [35]:
with open(csv_file, 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
    writer.writeheader()
    for data in resultados:
        writer.writerow(data)

### Generales

In [36]:
csv_file_general = 'resultados_algoritmos.csv'
csv_columns_general = ['Algoritmo', 'Tipo de Medición', 'Promedio General']

In [37]:
with open(csv_file_general, 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=csv_columns_general)
    writer.writeheader()
    for data in promedios_generales_resultados:
        writer.writerow(data)


## Resultados apartes

In [38]:
tiempos = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))


### Lectura de datos

In [39]:
with open('resultados_tiempos.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        palabra = row['palabra']
        algoritmo = row['metodo']
        tipo_medicion = row['medicion']
        tiempo = float(row['tiempo'])
        tiempos[palabra][algoritmo][tipo_medicion].append(tiempo)

### Preparar datos

In [40]:
resultados = defaultdict(list)
promedios_generales = defaultdict(lambda: defaultdict(float))

### Cálculos de promedios por palabra

In [41]:
for palabra, algoritmos in tiempos.items():
    for algoritmo, mediciones in algoritmos.items():
        for tipo_medicion, tiempos_lista in mediciones.items():
            promedio_tiempo = sum(tiempos_lista) / len(tiempos_lista)
            resultados[f'{algoritmo}_{tipo_medicion}'].append({
                'Palabra': palabra,
                'Algoritmo': algoritmo,
                'Tipo de Medición': tipo_medicion,
                'Promedio Tiempo': promedio_tiempo
            })
            # Acumular los tiempos para calcular el promedio general por algoritmo
            promedios_generales[algoritmo][tipo_medicion] += promedio_tiempo

### Exportar

In [42]:
csv_columns = ['Palabra', 'Algoritmo', 'Tipo de Medición', 'Promedio Tiempo']

In [43]:
for key, data_list in resultados.items():
    csv_file = f'resultados_{key}.csv'
    with open(csv_file, 'w', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        writer.writeheader()
        for data in data_list:
            writer.writerow(data)